# Pipeline WAV → MIDI → Partitura PDF

In [14]:
import crepe
import librosa
import pretty_midi
import numpy as np
from music21 import converter
from IPython.display import Audio, display

# === CONFIGURAÇÕES ===
audio_path = r"D:\Projetos\Melodyscribe\audios\one.wav"
midi_path = r"D:\Projetos\Melodyscribe\audios\one_crepe.mid"
xml_path = r"D:\Projetos\Melodyscribe\audios\one_crepe.musicxml"

# Parâmetros de suavização e filtro
MIN_NOTE_DURATION = 0.1      # segundos
SMOOTH_WINDOW_SIZE = 5       # frames (para média móvel)

# === 1. Carrega áudio ===
audio, sr = librosa.load(audio_path, sr=16000)
print("🔊 Escute o áudio original:")
display(Audio(audio, rate=sr))

# === 2. Estima pitch com CREPE ===
print("🎯 Estimando pitch com CREPE...")
time, frequency, confidence, activation = crepe.predict(audio, sr, viterbi=True)

# === 3. Suaviza o pitch com média móvel ===
def smooth_pitch(freq, window_size=5):
    return np.convolve(freq, np.ones(window_size)/window_size, mode='same')

frequency_smoothed = smooth_pitch(frequency, window_size=SMOOTH_WINDOW_SIZE)

# === 4. Constrói arquivo MIDI com notas filtradas ===
print("🎼 Convertendo pitch suavizado para MIDI...")
midi = pretty_midi.PrettyMIDI()
instrument = pretty_midi.Instrument(program=0)

frame_duration = time[1] - time[0]
note_on = None

for i, (f, conf) in enumerate(zip(frequency_smoothed, confidence)):
    if conf > 0.5 and f > 0:
        pitch = int(round(69 + 12 * np.log2(f / 440.0)))
        t = i * frame_duration
        if note_on is None:
            note_on = (pitch, t)
    else:
        if note_on:
            pitch, start = note_on
            end = i * frame_duration
            if end - start >= MIN_NOTE_DURATION:
                note = pretty_midi.Note(velocity=100, pitch=pitch, start=start, end=end)
                instrument.notes.append(note)
            note_on = None

# Fecha a última nota se necessário
if note_on:
    pitch, start = note_on
    end = len(frequency_smoothed) * frame_duration
    if end - start >= MIN_NOTE_DURATION:
        note = pretty_midi.Note(velocity=100, pitch=pitch, start=start, end=end)
        instrument.notes.append(note)

midi.instruments.append(instrument)
midi.write(midi_path)
print(f"✅ MIDI salvo em: {midi_path}")

# === 5. Converte para MusicXML (para abrir no MuseScore) ===
print("📄 Exportando para MusicXML...")
score = converter.parse(midi_path)
score.write("musicxml", fp=xml_path)
print(f"✅ MusicXML salvo em: {xml_path}")


🔊 Escute o áudio original:


🎯 Estimando pitch com CREPE...
85/85 ━━━━━━━━━━━━━━━━━━━━ 25s 296ms/step
🎼 Convertendo pitch suavizado para MIDI...
✅ MIDI salvo em: D:\Projetos\Melodyscribe\audios\one_crepe.mid
📄 Exportando para MusicXML...
✅ MusicXML salvo em: D:\Projetos\Melodyscribe\audios\one_crepe.musicxml
